<a href="https://colab.research.google.com/github/mitchelljphayes/COMP9444-group-assignment/blob/main/notebooks/mitch_fine_tune_llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade accelerate evaluate transformers datasets tiktoken torch rouge_score trl peft bitsandbytes

In [ ]:
!nvidia-smi
!nvcc --version

In [ ]:
import os
import math
import numpy as np
import torch
import evaluate
import transformers
from io import open
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoModelForCausalLM, Trainer, AutoTokenizer, TrainingArguments, logging, BitsAndBytesConfig
from transformers.integrations import TensorBoardCallback
from datasets import load_dataset

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
if torch.backends.mps.is_available():
    device = 'mps'

In [ ]:
print(device)
print(transformers.__version__)


In [ ]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()
if torch.backends.mps.is_available():
    torch.mps.empty_cache()

In [ ]:
project_root = '..'
dataset_name = 'vblagoje/lfqa_support_docs'
data_dir = os.path.join(project_root, 'data/lfqa/')
model_dir = os.path.join(project_root, 'models/')
# model_checkpoint = 'gpt2'
# model_checkpoint = 'distilgpt2'
# model_checkpoint = 'mistralai/Mistral-7B-v0.1'
model_checkpoint = 'NousResearch/Llama-2-7b-chat-hf'
# model_checkpoint = 'Salesforce/xgen-7b-8k-base'

In [ ]:
compute_dtype = getattr(torch, 'float16')

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)



In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, cache_dir=model_dir, quantization_config=quant_config, device_map={"":0})
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, cache_dir=model_dir, trust_remote_code=True)
tokenizer.padding_side = 'right'

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
model

In [ ]:
special_tokens_dict = {'eos_token': '<|endoftext|>'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# if not os.path.exists(data_dir):
#     os.mkdir(data_dir)
data = load_dataset(dataset_name, cache_dir=data_dir)

In [ ]:
data

In [ ]:
train_data = data['train']
val_data = data['validation']
# test_data = data['test']

In [ ]:
train_data

In [ ]:
train_data = train_data.shard(24, 0)
train_data

In [ ]:
def transform(example):
    qa = f"Question: {example['input']} \n Answer: {example['output'][0]['answer']}"
    max_length = 1024
    id_tensor = tokenizer.encode(qa, truncation=True, padding="max_length", max_length=max_length, return_tensors='np').squeeze(0)
    # id_tensor = tokenizer.encode(qa, return_tensors='np')
    return {"input_ids": id_tensor, "text": qa} ##"attention_mask": torch.ones(max_length)}

In [ ]:
train_data_tokz = train_data.map(transform, num_proc=4, remove_columns=["input", "output", "meta", "id"])
val_data_tokz = train_data.map(transform, num_proc=4, remove_columns=["input", "output", "meta", "id"])

In [ ]:
train_data_tokz

In [ ]:
example = train_data_tokz[5]['input_ids']
len(example)

In [ ]:
model.resize_token_embeddings(len(tokenizer))
# model = prepare_model_for_int8_training(model)
# model = get_peft_model(model, lora_peft_config)

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"../{model_dir}/{model_name}-finetuned-{dataset_name}",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    # per_device_eval_batch_size=4,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=100,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=3.0,
    max_steps=-1,
    warmup_ratio=0.3,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    # metric_for_best_model="eval_loss",
    # greater_is_better=False,
    # evaluation_strategy="epoch",
    # prediction_loss_only=True,
    # save_strategy="epoch",
)

In [ ]:
metric = evaluate.load("rouge")

In [ ]:
def compute_metrics(eval_pred):
    logits, past_keys = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=past_keys)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data_tokz,
    peft_config=peft_params,
    dataset_text_field='text',
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_args,
    packing=False,
    # eval_dataset=val_data_tokz,
    # compute_metrics=compute_metrics,
)

In [ ]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()
if torch.backends.mps.is_available():
    torch.mps.empty_cache()
import gc
del variables
gc.collect()

In [ ]:
trainer.train()
trainer.save_model(model_dir)